In [351]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.pipeline import Pipeline
from sklearn.datasets import load_wine

In [352]:
# dataSet 불러오기

wine = load_wine()

# 1. K-MEANS를 이용하여 Accuracy 측정

In [353]:
# K_MEANS
from sklearn.cluster import KMeans

model = KMeans(n_clusters=3,init='k-means++')
model.fit(wine.data,wine.target)
predict = pd.Series(model.predict(wine.data)) # predict로 iris['class']와 accuracy 측정
predict

0      1
1      1
2      1
3      1
4      0
      ..
173    0
174    0
175    0
176    0
177    2
Length: 178, dtype: int32

In [354]:
# K_MEANS crosstab -> 데이터를 재구조화 시키고 싶은데 하는 방법을 모르겠음
# 따라서 crosstab으로 나타낸 뒤 가장 많이 겹치는 쪽으로 매핑하는 방향으로

le = preprocessing.LabelEncoder() # 먼저 iris['class']를 형변환
target = pd.DataFrame(le.fit_transform(wine.target))

def mapping_func(target, predict): # 가장 정확도가 높은 mapping function
    predict_dict={}
    ct = pd.crosstab(target, predict)
    for i in range(0,len(ct),1):
        predict_dict[i] = ct.iloc[:,i].idxmax()
    return predict_dict

In [355]:
# 가장 정확도가 높은 쪽으로 매핑

predict_dict = mapping_func(target[0], predict)
predict = predict.map(predict_dict).astype('category')
predict

0      0
1      0
2      0
3      0
4      2
      ..
173    2
174    2
175    2
176    2
177    1
Length: 178, dtype: category
Categories (3, int64): [0, 1, 2]

In [356]:
# K_MEANS Accuracy Measurement
from sklearn.metrics import accuracy_score

accuracy_score(target, predict) # K_MEANS의 정확도는 약 70.22%

0.702247191011236

# 2. DBSCAN을 이용하여 Accuracy 측정

In [387]:
# DBSCAN
from sklearn.model_selection import train_test_split
from sklearn.cluster import DBSCAN

# create model and prediction
model = DBSCAN(eps=53) # 주변 밀집도 3으로 설정
model = model.fit(wine.data,wine.target)
predict = pd.Series(model.fit_predict(wine.data))

# predict_dict = mapping_func(target[0], predict)
# predict = predict.map(int,predict_dict) # 매핑함수를 적용하려 했지만, 하나의 값에 몰릴경우 어떻게 처리할지를 상정하지 않아서 값이 제대로 나오지 않음
ct = pd.crosstab(target[0], predict)
ct # CT으로 최대 정확도를 산정해 보면 70/178의 오답률을 보인다. (약 60.674% 정도)

col_0,-1,0,1
0,,,
0,7,39,13
1,1,1,69
2,0,0,48


# 3. SVM을 이용하여 Accuracy 측정

In [388]:
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split

trainX, testX, trainY, testY = \
    train_test_split(wine.data, wine.target, test_size=0.2)

svm = SVC(kernel='linear')
svm.fit(trainX, trainY)

svm.score(trainX,trainY) # (약 99.29%)

0.9929577464788732